In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import os

In [2]:
# get data files
PATH = '/Users/tunahanozcan/Desktop/machine-learning/book-recommendation-system/book-crossings'

books_filename = os.path.join(PATH, 'BX-Books.csv')
ratings_filename = os.path.join(PATH, 'BX-Book-Ratings.csv')

In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
# Merge datasets
df = pd.merge(df_ratings, df_books, on='isbn')
df

user        isbn  rating  \
0        276725  034545104X     0.0   
1        276726  0155061224     5.0   
2        276727  0446520802     0.0   
3        276729  052165615X     3.0   
4        276729  0521795028     6.0   
...         ...         ...     ...   
1031170  276704  0876044011     0.0   
1031171  276704  1563526298     9.0   
1031172  276706  0679447156     0.0   
1031173  276709  0515107662    10.0   
1031174  276721  0590442449    10.0   

                                                     title             author  
0                                     Flesh Tones: A Novel         M. J. Rose  
1                                         Rites of Passage         Judith Rae  
2                                             The Notebook    Nicholas Sparks  
3                                           Help!: Level 1      Philip Prowse  
4        The Amsterdam Connection : Level 4 (Cambridge ...        Sue Leather  
...                                                    ...                ...  
1031170  Edgar Cayce on the Akashic Records: The Book o...  Kevin J. Todeschi  
1031171  Get Clark Smart : The Ultimate Guide for the S...       Clark Howard  
1031172  Eight Weeks to Optimum Health: A Proven Progra...        Andrew Weil  
1031173   The Sherbrooke Bride (Bride Trilogy (Paperback))  Catherine Coulter  
1031174                                  Fourth Grade Rats     Jerry Spinelli  

[1031175 rows x 5 columns]

In [5]:
# Books with >=100 ratings
df_cleaned_books = df_ratings.groupby(["isbn"]).count().reset_index()
good_books = df_cleaned_books.loc[df_cleaned_books["rating"] >= 100]["isbn"]

# books contains those have no less than 100 ratings
good_books = df_books.loc[df_books["isbn"].isin(good_books)]

# Users with >=200 ratings
df_cleaned_users = df_ratings[["user", "rating"]].groupby(["user"]).count().reset_index()

good_users = df_cleaned_users.loc[df_cleaned_users["rating"] >= 200]["user"]

df = df_ratings.loc[df_ratings["user"].isin(good_users)]
df = df.loc[df["isbn"].isin(good_books["isbn"])]


In [6]:
# Pivot ratings into book specifications
df_book_features = df.pivot(
    index='isbn',
    columns='user',
    values='rating'
).fillna(0)

In [7]:
# Convert to sparse matrix
matrix_sparse = csr_matrix(df_book_features.values)


In [8]:
# Train KNN model
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(matrix_sparse)

NearestNeighbors(algorithm='brute', metric='cosine')

In [9]:
def get_recommends(title=""):
    try:
        book = good_books.loc[good_books["title"] == title]
    except KeyError as e:
        print('Book', e, 'Given book is not existed. Try another one!')
        return

    b = df_book_features.loc[df_book_features.index.isin(book["isbn"])]
    distance, indice = model.kneighbors([x for x in b.values], n_neighbors=6)

    distance = distance[0][1:]
    indice = indice[0][1:]

    titles = [
        df_books.loc[df_books['isbn'] == df_book_features.iloc[i].name]["title"].values[0]
        for i in indice
    ]

    recommended = [[t, float(f"{d:.2f}")] for t, d in zip(titles, distance)][::-1]
    return [title, recommended]


In [10]:
print(get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))"))

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8], ['The Weight of Water', 0.77], ['The Surgeon', 0.77], ['I Know This Much Is True', 0.77], ['The Lovely Bones: A Novel', 0.72]]]


In [11]:
# Test the KNN and get_recommends function is working properly
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8], ['The Weight of Water', 0.77], ['The Surgeon', 0.77], ['I Know This Much Is True', 0.77], ['The Lovely Bones: A Novel', 0.72]]]
You passed the challenge! 🎉🎉🎉🎉🎉
